Load all datasets

In [2]:
from google.colab import files

uploaded = files.upload()

Saving cleaned_cropdata.csv to cleaned_cropdata.csv
Saving ndvi_regions.csv to ndvi_regions.csv
Saving satellite_rain_temp.csv to satellite_rain_temp.csv


In [4]:
import pandas as pd

# Load files
df_main = pd.read_csv("cleaned_cropdata.csv")
ndvi_df = pd.read_csv("ndvi_regions.csv")
weather_df = pd.read_csv("satellite_rain_temp.csv")

In [5]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128544 entries, 0 to 128543
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Rainfall      128544 non-null  float64
 1   Temperature   128544 non-null  float64
 2   Ph            128544 non-null  float64
 3   Crop          128544 non-null  object 
 4   Production    128544 non-null  float64
 5   Crop_encoded  128544 non-null  int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 5.9+ MB


In [6]:
import numpy as np

# Define the same 7 regions used in your satellite pulls
regions = ["Kunene", "Oshana", "Kavango West", "Zambezi", "Omaheke", "Hardap", "Erongo"]

# Define month labels from Jan 2022 to July 2025
months = pd.date_range(start='2022-01-01', end='2025-07-01', freq='MS').to_period('M').astype(str)

# Add synthetic region and month
np.random.seed(42)
df_main['region'] = np.random.choice(regions, size=len(df_main))
df_main['month'] = np.random.choice(months, size=len(df_main))

print("Synthetic 'region' and 'month' added")
print(df_main[['region', 'month']].head())

Synthetic 'region' and 'month' added
         region    month
0        Erongo  2024-11
1       Zambezi  2025-06
2       Omaheke  2025-05
3        Erongo  2024-03
4  Kavango West  2023-03


In [7]:
# Extract month from NDVI 'date' column
ndvi_df['date'] = pd.to_datetime(ndvi_df['date'])
ndvi_df['month'] = ndvi_df['date'].dt.to_period('M').astype(str)

# Merge main + NDVI
merged = pd.merge(df_main, ndvi_df[['region', 'month', 'NDVI']], on=['region', 'month'], how='left')

# Merge with rainfall & temp
merged = pd.merge(merged, weather_df, on=['region', 'month'], how='left')

print("🔍 Merged shape:", merged.shape)
print(merged.head())
print(merged.shape)

🔍 Merged shape: (164374, 11)
    Rainfall  Temperature   Ph   Crop  Production  Crop_encoded        region  \
0  423.38549         27.0  4.3  Bajra     0.77952             0        Erongo   
1  423.40081         27.0  4.3  Bajra     0.78003             0       Zambezi   
2  423.42756         27.0  4.3  Bajra     0.78092             0       Omaheke   
3  423.43750         27.0  4.3  Bajra     0.78125             0        Erongo   
4  423.45455         27.0  4.3  Bajra     0.78182             0  Kavango West   

     month  NDVI  rainfall_mm     temp_C  
0  2024-11   NaN     0.193167  27.944364  
1  2025-06   NaN          NaN        NaN  
2  2025-05   NaN     0.160896  22.753183  
3  2024-03   NaN     0.766192  31.347102  
4  2023-03   NaN     1.658625  31.491290  
(164374, 11)


In [8]:
print(merged.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164374 entries, 0 to 164373
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Rainfall      164374 non-null  float64
 1   Temperature   164374 non-null  float64
 2   Ph            164374 non-null  float64
 3   Crop          164374 non-null  object 
 4   Production    164374 non-null  float64
 5   Crop_encoded  164374 non-null  int64  
 6   region        164374 non-null  object 
 7   month         164374 non-null  object 
 8   NDVI          77733 non-null   float64
 9   rainfall_mm   158490 non-null  float64
 10  temp_C        158490 non-null  float64
dtypes: float64(7), int64(1), object(3)
memory usage: 13.8+ MB
None


Clean merged data

In [9]:
# Fill NDVI, rainfall_mm, and temp_C with regional means
merged['NDVI'] = merged.groupby('region')['NDVI'].transform(lambda x: x.fillna(x.mean()))
merged['rainfall_mm'] = merged.groupby('region')['rainfall_mm'].transform(lambda x: x.fillna(x.mean()))
merged['temp_C'] = merged.groupby('region')['temp_C'].transform(lambda x: x.fillna(x.mean()))

# Check if anything is still missing
print("Missing values remaining:\n", merged.isnull().sum())

# Save the cleaned version
merged.to_csv("merged_data_cleaned.csv", index=False)
print("Cleaned data saved as 'merged_data_cleaned.csv'")


Missing values remaining:
 Rainfall        0
Temperature     0
Ph              0
Crop            0
Production      0
Crop_encoded    0
region          0
month           0
NDVI            0
rainfall_mm     0
temp_C          0
dtype: int64
Cleaned data saved as 'merged_data_cleaned.csv'


In [18]:
crop_rename_map = {
    "Bajra": "Pearl Millet",
    "Brinjal": "Egg Plant",
    "Jowar": "Sorghum",
    "Jute": "Golden Fiber",
    "Khesari": "Grass Pea",
    "Mesta": "Fiber",
    "Moong(Green Gram)": "Green Gram",
    "Ragi": "Finger Millet",
    "Urad": "Black Gram"
}

In [19]:
merged['Crop'] = merged['Crop'].replace(crop_rename_map)

In [20]:
merged['Crop'].unique()

array(['Pearl Millet', 'Banana', 'Barley', 'Bean', 'Blackgram',
       'Egg Plant', 'Castor seed', 'Chillies', 'Coriander', 'Cotton',
       'Cowpea', 'Drum Stick', 'Garlic', 'Gram', 'Grapes', 'Groundnut',
       'Guar seed', 'Horse-gram', 'Sorghum', 'Golden Fiber', 'Grass Pea',
       'Lady Finger', 'Lentil', 'Linseed', 'Maize', 'Fiber', 'Green Gram',
       'Moth', 'Onion', 'Orange', 'Peas & beans (Pulses)', 'Potato',
       'Raddish', 'Finger Millet', 'Rice', 'Safflower', 'Sannhamp',
       'Sesamum', 'Soyabean', 'Sugarcane', 'Sunflower', 'Sweet potato',
       'Tapioca', 'Tomato', 'Black Gram', 'Wheat'], dtype=object)

In [21]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164374 entries, 0 to 164373
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Rainfall      164374 non-null  float64
 1   Temperature   164374 non-null  float64
 2   Ph            164374 non-null  float64
 3   Crop          164374 non-null  object 
 4   Production    164374 non-null  float64
 5   Crop_encoded  164374 non-null  int64  
 6   region        164374 non-null  object 
 7   month         164374 non-null  object 
 8   NDVI          164374 non-null  float64
 9   rainfall_mm   164374 non-null  float64
 10  temp_C        164374 non-null  float64
dtypes: float64(7), int64(1), object(3)
memory usage: 13.8+ MB


In [22]:
files.download("merged_data_cleaned.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>